In [18]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
from math import *
import os, shutil
import glob
import fitz
import re
from PIL import Image,ImageFont,ImageDraw
from pandas.core.frame import DataFrame
from openpyxl.writer.excel import ExcelWriter
import math
import PyPDF2
import sys



In [19]:
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:  # 判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)  # makedirs 创建文件时如果路径不存在会创建这个路径
    else:
        pass

In [20]:
excelname = '15032023'
pdfname = '15032023'
#pdfname1 = '09032023_1'
resimg_path = './res-image/' + pdfname
used_img_path = "./used-sousimg/" + pdfname
used_img_UK_path = './used_sousimg_UK/' + pdfname
mkdir(resimg_path)
mkdir(used_img_path)
mkdir(used_img_UK_path)
mkdir('./pdf_res/' + pdfname )

In [21]:
def order_FR(clientData,numero_suivi):
    '''
        用于生成目的地为FR的邮票
        Args:
            clientData: list,包含一个客户的信息，以及订单号
            numero_suivi: string,一张邮票的SD编号,也是货运编号,也是这张邮票的名称
        Returns:
    '''
    timbrePath = './sousimgs/' + numero_suivi
    # 拼接图片

    order = Image.new("RGB",(1000,1000),(255,255,255))     #生成1000*1000px 白底图片，
    timbre = Image.open(timbrePath + '.jpg').resize((581,305),Image.Resampling.BILINEAR)  # 读取timbre
    order.paste(timbre,(355,0))     # 贴上邮票

    # 开始处理客人信息，先将客人信息整合到一起，制作成一张600*260 的小图
    # 然后贴到order这张大图中
    address2 =  clientData[5] if not pd.isna(clientData[5]) else '' 
    info_client = str(clientData[3]) + '\n' + str(clientData[4]) +'\n' + str(address2) + '\n' + str(clientData[6]) + '  ' + str(clientData[7]) + '\n' + str(clientData[8])
    
    img_info = Image.new("RGB",(600,260),(255,255,255))
    dr = ImageDraw.Draw(img_info)
    font = ImageFont.truetype(os.path.join("C:/Windows/Fonts/times.TTF"),28)
    dr.text( (0,0), info_client, font=font, fill="#000000")
    order.paste(img_info,(400,350)) 

    # 开始处理我们的信息，同客人信息一样
    infopreciousse="            Expéditeur : "+'\n'
    infopreciousse+=" INFINITUS - PRECIOUSSE"+'\n'
    infopreciousse+="5 BIS RUE DE SAINT-GOBAIN"+'\n'
    infopreciousse+="    CENTRE CIFA LOT3211 "+'\n'
    infopreciousse+="     93300 AUBERVILLIERS "+'\n'
    im3 = Image.new("RGB", (350,260), (255, 255, 255))
    dr2 = ImageDraw.Draw(im3)
    font = ImageFont.truetype(os.path.join("C:/Windows/Fonts/ARIAL.TTF"), 16)
    ##第一个参数是前后，第二个参数是上下
    dr2.text((50, 0), infopreciousse, font=font, fill="#000000")
    box = (0,80)
    order.paste(im3,box)

    # 这一步是裁剪已经完成的order面单，得合适的大小
    box = (0,0,500,1000)
    order = order.rotate(90)
    order = order.crop(box)

    order.save(resimg_path+'/' + str(clientData[1]) + '.pdf')

    shutil.move(timbrePath + '.jpg', used_img_path)

In [22]:
"""def order_UK(clientData,numero_suivi):
    '''
        用于生成目的地为UK的邮票,这情况有点特殊在于，他的timbre是分为两张图片的的，且存于sousimgs_UK文件中
        Args:
            clientData: list,包含一个客户的信息，以及订单号
            numero_suivi: string,一张邮票的SD编号,也是货运编号,也是这张邮票的名称
        Returns:
    '''
    timbre_dir = './sousimgs_UK'
    timbre_name = [ f for f in os.listdir(timbre_dir) if f.startswith(numero_suivi)]
    timbrePath1 = timbre_dir +'/' + timbre_name[0]
    timbrePath2 = timbre_dir + '/' + timbre_name[1]
    timbre1 = Image.open(timbrePath1).resize((500,350),Image.Resampling.BILINEAR)  # 读取timbre1
    timbre2 = Image.open(timbrePath2).resize((500,350),Image.Resampling.BILINEAR)   # 读取timbre2
    order = Image.new("RGB",(1000,1000),(255,255,255))     #生成1000*1000px 白底图片，
    order.paste(timbre1,(0,0))     # 贴上timbre1
    order.paste(timbre2,(500,0))     # 贴上timbre2


    # 开始处理客人信息，先将客人信息整合到一起，制作成一张600*260 的小图
    # 然后贴到order这张大图中
    address2 =  clientData[5] if not pd.isna(clientData[5]) else ''
    info_client = clientData[3] + '\n' + clientData[4] +'\n' + address2 + '\n' + str(clientData[6]) + '  ' + clientData[7] + '\n' + clientData[8]
    img_info = Image.new("RGB",(600,260),(255,255,255))
    dr = ImageDraw.Draw(img_info)
    font = ImageFont.truetype(os.path.join("C:/Windows/Fonts/times.TTF"),28)
    dr.text( (0,0), info_client, font=font, fill="#000000")
    order.paste(img_info,(610,350)) 

    # 开始处理我们的信息，同客人信息一样
    infopreciousse="            Expéditeur : "+'\n'
    infopreciousse+=" INFINITUS - PRECIOUSSE"+'\n'
    infopreciousse+="5 BIS RUE DE SAINT-GOBAIN"+'\n'
    infopreciousse+="    CENTRE CIFA LOT3211 "+'\n'
    infopreciousse+="     93300 AUBERVILLIERS "+'\n'
    im3 = Image.new("RGB", (400,300), (255, 255, 255))
    dr2 = ImageDraw.Draw(im3)
    font = ImageFont.truetype(os.path.join("C:/Windows/Fonts/ARIAL.TTF"), 18)
    ##第一个参数是前后，第二个参数是上下
    dr2.text((50, 0), infopreciousse, font=font, fill="#000000")
    box = (50,350)
    order.paste(im3,box)
    box = (0,0,500,1000)
    order = order.rotate(90)
    order = order.crop(box)
    order.save(resimg_path+'/' + str(clientData[1]) + '.pdf')
    shutil.move(timbrePath1,used_img_UK_path)
    shutil.move(timbrePath2,used_img_UK_path)"""


'def order_UK(clientData,numero_suivi):\n    \'\'\'\n        用于生成目的地为UK的邮票,这情况有点特殊在于，他的timbre是分为两张图片的的，且存于sousimgs_UK文件中\n        Args:\n            clientData: list,包含一个客户的信息，以及订单号\n            numero_suivi: string,一张邮票的SD编号,也是货运编号,也是这张邮票的名称\n        Returns:\n    \'\'\'\n    timbre_dir = \'./sousimgs_UK\'\n    timbre_name = [ f for f in os.listdir(timbre_dir) if f.startswith(numero_suivi)]\n    timbrePath1 = timbre_dir +\'/\' + timbre_name[0]\n    timbrePath2 = timbre_dir + \'/\' + timbre_name[1]\n    timbre1 = Image.open(timbrePath1).resize((500,350),Image.Resampling.BILINEAR)  # 读取timbre1\n    timbre2 = Image.open(timbrePath2).resize((500,350),Image.Resampling.BILINEAR)   # 读取timbre2\n    order = Image.new("RGB",(1000,1000),(255,255,255))     #生成1000*1000px 白底图片，\n    order.paste(timbre1,(0,0))     # 贴上timbre1\n    order.paste(timbre2,(500,0))     # 贴上timbre2\n\n\n    # 开始处理客人信息，先将客人信息整合到一起，制作成一张600*260 的小图\n    # 然后贴到order这张大图中\n    address2 =  clientData[5] if not pd.isna(clientD

In [23]:
def order_UK(clientData,numero_suivi):
    '''
        用于生成目的地为ES的邮票,这情况有点特殊在于，他的timbre是分为两张图片的的，且存于sousimgs_UK文件中
        和UK是同一个fonction，只不过重命名了
        Args:
            clientData: list,包含一个客户的信息，以及订单号
            numero_suivi: string,一张邮票的SD编号,也是货运编号,也是这张邮票的名称
        Returns:
    '''
    timbre_dir = './sousimgs_UK'
    timbre_name = [ f for f in os.listdir(timbre_dir) if f.startswith(numero_suivi)]
    timbrePath1 = timbre_dir +'/' + timbre_name[0]
    timbrePath2 = timbre_dir + '/' + timbre_name[1]
    timbre1 = Image.open(timbrePath1).resize((380,270),Image.Resampling.BILINEAR)  # 读取timbre1
    timbre2 = Image.open(timbrePath2).resize((380,270),Image.Resampling.BILINEAR)   # 读取timbre2
    order = Image.new("RGB",(1000,1000),(255,255,255))     #生成1000*1000px 白底图片，
    order.paste(timbre1,(250,0))     # 贴上timbre1
    order.paste(timbre2,(630,0))     # 贴上timbre2


    # 开始处理客人信息，先将客人信息整合到一起，制作成一张600*260 的小图
    # 然后贴到order这张大图中
   
    address2 =  clientData[5] if not pd.isna(clientData[5]) else ''
   # info_client = str(clientData[3]) + '\n' + clientData[4] +'\n' + str(address2) + '\n' + str(clientData[6]) + '  ' + clientData[7] + '\n' + clientData[8]
    info_client = str(clientData[3]) + '\n' + str(clientData[4]) +'\n' + str(address2) + '\n' + str(clientData[6]) + '  ' + str(clientData[7]) + '\n' + str(clientData[8])
  
    img_info = Image.new("RGB",(600,250),(255,255,255))
    dr = ImageDraw.Draw(img_info)
    font = ImageFont.truetype(os.path.join("C:/Windows/Fonts/times.TTF"),28)
    dr.text( (0,0), info_client, font=font, fill="#000000")
    order.paste(img_info,(380,300)) 

    # 开始处理我们的信息，同客人信息一样
    infopreciousse="            Expéditeur : "+'\n'
    infopreciousse+=" INFINITUS - PRECIOUSSE"+'\n'
    infopreciousse+="5 BIS RUE DE SAINT-GOBAIN"+'\n'
    infopreciousse+="    CENTRE CIFA LOT3211 "+'\n'
    infopreciousse+="     93300 AUBERVILLIERS "+'\n'
    infopreciousse+="           FRANCE"
    im3 = Image.new("RGB", (250,200), (255, 255, 255))
    dr2 = ImageDraw.Draw(im3)
    font = ImageFont.truetype(os.path.join("C:/Windows/Fonts/ARIAL.TTF"), 14)
    ##第一个参数是前后，第二个参数是上下
    dr2.text((50, 0), infopreciousse, font=font, fill="#000000")
    box = (3,50)
    order.paste(im3,box)
    box = (0,0,500,1000)
    order = order.rotate(90)
    order = order.crop(box)
    order.save(resimg_path+'/' + str(clientData[1]) + '.pdf')
    shutil.move(timbrePath1,used_img_UK_path)
    shutil.move(timbrePath2,used_img_UK_path)


In [24]:
def sep_pdf3(pdfname):
    pdf_reader = PyPDF2.PdfReader(pdfname + '.pdf')
    for pg in pdf_reader.pages:
        #num_preciousse = re.findall(r"(?:Réf\. client :|Réf desti :|DESTINATAIRERéf Client : )\s*\d{4}",pg.extract_text())
        num_aprizo = []
        num_aprizo = re.findall(r"APRIZO\d{4}",pg.extract_text())
        if not num_aprizo:
            num_aprizo = re.findall(r'Aprizo\d{4}',pg.extract_text())
        order_str = ''
        order_str = order_str + num_aprizo[0]
        pdf_writer = PyPDF2.PdfWriter();
        pdf_writer.add_page(pg)
        pdf_writer.write('./pdf_res/' + pdfname + '/' + order_str + '.pdf')

In [ ]:
########################################################################
###################       切割pdf          #############################
########################################################################
sep_pdf3(pdfname)

In [27]:
def colissimo(clientdata,pdfname):
    '''
        用于订单是colissimo的情况，这个情况的话是不需要生成lettre_suivi的面单
        只是需要在pdf_res文件中匹配到该订单对应的货运单，然后将该运单面移入res-img
        注意要修改一下货运单的名称，要以订单编号命名
        并且要返回货运单号，Num de coli
        Args: 
            clientdata: list,某一个订单的信息，包括订单编号等
            pdfname: string,要是使用的pdf名称，用来在pdf_res文件里进行定位单张订单
        Returns:
            Numéro de coli
    '''
    file_list = []
    src_path = './pdf_res/' + pdfname
    dst_path = './res-image/' + pdfname
    for items in os.walk(src_path, topdown=False):
        file_list = [item[:-4] for item in items[2]]  #读取出每个pdf的名字，并且删掉 .pdf 的后缀
    numero_coli = ''
    for f in file_list:
        if(clientdata[2].lower() == f.lower()):
            pdf_reader = PyPDF2.PdfReader(src_path + '/'+ f + '.pdf')
           # text = [pg.extract_text() for pg in pdf_reader.pages]
            text = pdf_reader.pages[0].extract_text()
            tmp = re.findall(r"(?:6A \d{10} \d|CA \d{4} \d{4} \d FR|6M \d{5} \d{5} \d|9L \d{10} \d)",text)
            if tmp:
                tmp[0] = tmp[0].replace(" ","")
            if not tmp:
                text = text.replace('\n',' ')
                tmp =re.findall(r'N° d\'expédition (\d{8})',text)
            numero_coli = tmp[0]
            shutil.move(src_path + '/'+ f + '.pdf' , dst_path)
            os.rename(dst_path + '/' + f + '.pdf', dst_path +'/'+ str(clientdata[1]) + '.pdf')
            break
    
    return numero_coli

In [28]:

df = pd.read_excel(excelname+'.xlsx')
df.loc[:,'Exception'] = None
df2 = pd.DataFrame(columns=['订单编号*','物流名称','物流渠道','货运单号'])
str_lettre = 'lettre suivie'
nb_lettre = 0
nb_lettre_uk =0
nb_lettre_es = 0
for i in df.iloc[:,0]:
    if re.search(str_lettre,i,re.IGNORECASE):
        nb_lettre+=1
    elif 'FR' in i:
        nb_lettre+=1
    elif 'ES' in i:
        nb_lettre_es+=1
    elif 'UK' in i:
        nb_lettre_uk+=1
sumline = nb_lettre + nb_lettre_uk + nb_lettre_es
sousimg_dir = './sousimgs/'
src_path = './sousimgs_UK/'
list_timbre = []
list_timbre_UK = []
#   这一步在于判读邮票数量是否还充足 
#   sousimgs文件中就是所有的邮票数
for items in os.walk(sousimg_dir, topdown=False):
    list_timbre = items[2]
list_timbre = [ timbre[:-4] for timbre in list_timbre]

for items in os.walk(src_path,topdown=False):
    list_timbre_UK = items[2]
list_timbre_UK = [timbre[:-4] for timbre in list_timbre_UK]

if(nb_lettre>len(list_timbre)):
    print("订单数大于邮票数,请先补充邮票数量，如果想继续运行程序，请注释这句代码下面 sys.exit()代码再次运行 ")
    sys.exit()
else:
    print("预计运行完成后剩余%d张图片可以使用"%(len(list_timbre)-nb_lettre))

预计运行完成后剩余179张图片可以使用


In [29]:
'''
整个程序要处理的是 
根据mabang导出的订单excel,利用其中的客户信息和sousimgs中的单张邮票来生成
可以直接打印的lettre suivi面单。
根据excel中的“买家自选物流方式”列, 一共有4种不同的订单情况要处理
case1: Std FR, 订单运输方式就是普通的lettre suivi
case2: Std UK,订单运输方式为lettre suivi UK
case3: Lettre suivie,订单运输方式为普通lettre suivi
case4: colissimo,订单运输方式为colissimo的
case5: ES,西班牙的
exception: 如果不是以上四种情况,在excel中添加一列 名称为 excption

生成的面单放入 res-image/excelname文件中,使用过的单张邮票放入used-sousimg/excelname中

如果这个主程序要封装成函数的话，要提供什么的para？ 我觉得应该有excelname 和 pdfname，虽然这两个的值是一样的
然后就是文件开头定义的那几个全局变量，涉及到文件的位置，所以也要修改相应的函数
'''
index_timbre = 0
index_timbre_UK = 0
for index,row in df.iterrows():
    numero_coli = None
    client_data = list(row)
    point_relay = 1
    if 'fr' in row[0].lower():
        numero_coli = list_timbre[index_timbre]
        order_FR(client_data,numero_coli)
        index_timbre+=1
    elif 'uk' in row[0].lower():
        numero_coli = list_timbre_UK[index_timbre_UK]
        order_UK(client_data,numero_coli )
        index_timbre_UK +=2
    elif 'Lettre suivie'.lower() in row[0].lower():
        numero_coli = list_timbre[index_timbre]
        order_FR(client_data, numero_coli)
        index_timbre+=1
    elif 'Colissimo'.lower() in row[0].lower():
        numero_coli = colissimo(client_data,pdfname)
    elif 'point de retrait' in row[0].lower():
        numero_coli = colissimo(client_data,pdfname)
        point_relay = 2
    elif 'es' in row[0].lower():
        numero_coli = list_timbre_UK[index_timbre_UK]
        order_UK(client_data,numero_coli )
        index_timbre_UK +=2
    elif 'it' in row[0].lower():
        numero_coli = list_timbre_UK[index_timbre_UK]
        order_UK(client_data,numero_coli )
        index_timbre_UK +=2
    else:
        df.loc[index,'Exception'] = '这个订单出异常了，要检查'
    df.loc[index,'货运单号'] = numero_coli
    df2.loc[index,'货运单号'] = numero_coli
    df2.loc[index,'订单编号*'] = df.loc[index,'订单编号']
    if point_relay == 1 :
        df2.loc[index,'物流名称'] = 'Colissimo'
        df2.loc[index,'物流渠道'] = 'Colissimo'
    else:
        df2.loc[index,'物流名称'] = 'Mondial Relay'
        df2.loc[index,'物流渠道'] = 'Mondial Relay'

df.to_excel(excelname+'_new.xlsx',index=False,sheet_name='Orders')
df2.to_excel(excelname+'_mabang.xlsx',index=False,sheet_name='Orders')

        
        







In [30]:
##################################################################################
##########################         LOG                ############################
##################################################################################
#08/03/2023

"""1.修改判断邮票数量是否充足，因为提供的excel中有很多订单是colissimo的，修改如下
  原代码
    df = pd.read_excel(excelname+'.xlsx')
    sumline = len(df)
  修改后的
    df = pd.read_excel(excelname+'.xlsx')
str_lettre = 'lettre suivie'
nb_lettre = 0
for i in df.iloc[:,0]:
    if re.search(str_lettre,i,re.IGNORECASE):
        nb_lettre+=1
    elif 'FR' in i:
        nb_lettre+=1
    elif 'ES' in i:
        nb_lettre+=1
    elif 'UK' in i:
        nb_lettre+=1
print(nb_lettre)"""
"""2. 修改了邮票大小"""
"""3.对于point de retrait类型的，修改了货运单号的查找逻辑"""

"""df = pd.read_excel(excelname+'.xlsx')
data = list(df.iloc[18])
print(data)

order_ES(data,'87000746565992V')"""
'''
df = pd.read_excel(excelname+'.xlsx')
data = list(df.iloc[14])
colissimo(data,pdfname)
'''


# 15/03/2023 
'''
    修改查找  Aprizo****  字符串的逻辑，主要的修改处为function sep_pdf()
    这是修改之后被弃用的sep_pdf()
########################################################################
###################       切割pdf          #############################
########################################################################
"""
def sep_pdf(pdfname):
    pdf_reader = PyPDF2.PdfReader(pdfname + '.pdf')
    for pg in pdf_reader.pages:
        num_preciousse = re.findall(r"(?:Réf\. client :|Réf desti :|DESTINATAIRERéf Client : )\s*\d{4}",pg.extract_text())
        order_str = "APRIZO" + str(num_preciousse[0][-4:])
        pdf_writer = PyPDF2.PdfWriter();
        pdf_writer.add_page(pg)
        pdf_writer.write('./pdf_res/' + pdfname + '/' + order_str + '.pdf')
def sep_pdf2(pdfname,pdfname1):
    pdf_reader = PyPDF2.PdfReader(pdfname1 + '.pdf')
    for pg in pdf_reader.pages:
        #num_preciousse = re.findall(r"(?:Réf\. client :|Réf desti :|DESTINATAIRERéf Client : )\s*\d{4}",pg.extract_text())
        num_aprizo = re.findall(r"APRIZO\d{4}",pg.extract_text())
        order_str = ''
        for a in num_aprizo:
            order_str = order_str + a
        pdf_writer = PyPDF2.PdfWriter();
        pdf_writer.add_page(pg)
        pdf_writer.write('./pdf_res/' + pdfname + '/' + order_str + '.pdf')
        
使用方法，这里是处理colissimo和mondiale的pdf的地方
如果有多个mondiale的pdf要处理，那就需要多次执行此代码块，请按照以下顺序来执行
1.第一次执行此代码块，代码情况应该如下的，如果不是请修改为这样：
    sep_pdf(pdfname)
    #pdfname1 = '09032023_2'
    sep_pdf2(pdfname,pdfname1)
  第一次执行是处理了colissimo和第一个mondiale的pdf，如果还需要处理其余的mondiale的pdf，请执行第二步
2. 处理第二个mondiale的pdf代码要修改，要把 pdfname1 变量的值修改为第二个mondiale的pdf的名字，并且删掉
   pdfname1前的注释符号#，然后给sep_pdf(pdfname)添加注释符号，例子如下
    #sep_pdf(pdfname)
    pdfname1 = '09032023_2'
    sep_pdf2(pdfname,pdfname1)
    然后再次执行此代码块
    如果还有mondiale的pdf需要处理，继续修改pdfname1的值再执行代码块即可
3.处理完所有的pdf之后，复原代码块如下面的模样，也就是恢复到第一次的模样
    sep_pdf(pdfname)
    #pdfname1 = '09032023_2'
    sep_pdf2(pdfname,pdfname1)

"""

#sep_pdf(pdfname)
#pdfname1 = '09032023_2'
#sep_pdf2(pdfname,pdfname1)

'''

'\n    修改查找  Aprizo****  字符串的逻辑，主要的修改处为function sep_pdf()\n    这是修改之后被弃用的sep_pdf()\n########################################################################\n###################       切割pdf          #############################\n########################################################################\n"""\ndef sep_pdf(pdfname):\n    pdf_reader = PyPDF2.PdfReader(pdfname + \'.pdf\')\n    for pg in pdf_reader.pages:\n        num_preciousse = re.findall(r"(?:Réf\\. client :|Réf desti :|DESTINATAIRERéf Client : )\\s*\\d{4}",pg.extract_text())\n        order_str = "APRIZO" + str(num_preciousse[0][-4:])\n        pdf_writer = PyPDF2.PdfWriter();\n        pdf_writer.add_page(pg)\n        pdf_writer.write(\'./pdf_res/\' + pdfname + \'/\' + order_str + \'.pdf\')\ndef sep_pdf2(pdfname,pdfname1):\n    pdf_reader = PyPDF2.PdfReader(pdfname1 + \'.pdf\')\n    for pg in pdf_reader.pages:\n        #num_preciousse = re.findall(r"(?:Réf\\. client :|Réf desti :|DESTINATAIRERéf Client : )\\s*\\d{4}",pg.ex